Imports.

In [32]:
from enum import Enum
from typing import NamedTuple
import numpy as np
import math
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Input, UpSampling2D, ZeroPadding2D, \
                         Cropping2D, Reshape, Layer
import keras
from keras import backend as K
from skimage.filters import gabor_kernel
from keras.utils import conv_utils
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.engine.base_layer import InputSpec
from keras.legacy import interfaces
import tensorflow as tf

class _ConvX(Layer):
    def __init__(self,
                 filters,
                 kernel_size,
                 strides=1,
                 padding='valid',
                 data_format=None,
                 dilation_rate=1,
                 activation=None,
                 use_bias=False,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(_ConvX, self).__init__(**kwargs)
        self.rank = 2
        self.filters = filters
        self.kernel_size = conv_utils.normalize_tuple(kernel_size, 2,
                                                      'kernel_size')
        self.strides = conv_utils.normalize_tuple(strides, 2, 'strides')
        self.padding = conv_utils.normalize_padding(padding)
        self.data_format = K.normalize_data_format(data_format)
        self.dilation_rate = conv_utils.normalize_tuple(dilation_rate, 2,
                                                        'dilation_rate')
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(ndim=self.rank + 2)

    def build(self, input_shape):
        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1
        if input_shape[channel_axis] is None:
            raise ValueError('The channel dimension of the inputs '
                             'should be defined. Found `None`.')
        input_dim = input_shape[channel_axis]
        kernel_shape = self.kernel_size + (input_dim, self.filters)

        self.input_dim = input_dim #get the number of 2D grids in the input tensor

        self.kernel = self.add_weight( kernel_shape, #( self.filters, 2 ),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)

        self.bias = None
        # Set input spec.
        self.input_spec = InputSpec(ndim=self.rank + 2,
                                    axes={channel_axis: input_dim})

        gaborKernelStackShape = list( self.kernel_size )
        gaborKernelStackShape.append( self.input_dim ) #self.input_dim is the number of 2D grids in the input tensor
        gaborKernelStackShape.append( self.filters ) #self.filters is the number of 2D grids in the output tensor
        gaborKernels = np.zeros( tuple( gaborKernelStackShape ) )
        self.gaborKernelsTensor = K.variable(value=gaborKernels, dtype=K.dtype( self.kernel ), name='gaborKernelsTensor') 


        self.built = True

    def call(self, inputs):

        zeros = np.zeros( (1,1,1,40) )
        zerosTensor = K.variable(value=zeros, dtype=K.dtype( self.kernel ), name='zerosTensor') 
        outputs = K.conv2d(
                inputs,
                self.gaborKernelsTensor, 
                strides=self.strides,
                padding=self.padding,
                data_format=self.data_format,
                dilation_rate=self.dilation_rate)
        
        outputs *= 1 + 0*K.conv2d(
                zerosTensor,
                self.kernel, 
                strides=self.strides,
                padding=self.padding,
                data_format=self.data_format,
                dilation_rate=self.dilation_rate)

        if self.activation is not None:
            out = self.activation(outputs)
            return out
        return outputs

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            space = input_shape[1:-1]
        elif self.data_format == 'channels_first':
            space = input_shape[2:]
        new_space = []
        for i in range(len(space)):
            new_dim = conv_utils.conv_output_length(
                space[i],
                self.kernel_size[i],
                padding=self.padding,
                stride=self.strides[i],
                dilation=self.dilation_rate[i])
            new_space.append(new_dim)
        if self.data_format == 'channels_last':
            return (input_shape[0],) + tuple(new_space) + (self.filters,)
        elif self.data_format == 'channels_first':
            return (input_shape[0], self.filters) + tuple(new_space)

    def get_config(self):
        config = {
            'rank': self.rank,
            'filters': self.filters,
            'kernel_size': self.kernel_size,
            'strides': self.strides,
            'padding': self.padding,
            'data_format': self.data_format,
            'dilation_rate': self.dilation_rate,
            'activation': activations.serialize(self.activation),
            'use_bias': self.use_bias,
            'kernel_initializer': initializers.serialize(self.kernel_initializer),
            'bias_initializer': initializers.serialize(self.bias_initializer),
            'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
            'bias_regularizer': regularizers.serialize(self.bias_regularizer),
            'activity_regularizer':
                regularizers.serialize(self.activity_regularizer),
            'kernel_constraint': constraints.serialize(self.kernel_constraint),
            'bias_constraint': constraints.serialize(self.bias_constraint)
        }
        base_config = super(_ConvX, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    

class Conv2DX(_ConvX):
    @interfaces.legacy_conv2d_support
    def __init__(self, filters,
                 kernel_size,
                 strides=(1, 1),
                 padding='valid',
                 data_format=None,
                 dilation_rate=(1, 1),
                 activation=None,
                 use_bias=False,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(Conv2DX, self).__init__(
            filters=filters,
            kernel_size=kernel_size,
            strides=strides,
            padding=padding,
            data_format=data_format,
            dilation_rate=dilation_rate,
            activation=activation,
            use_bias=False,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            **kwargs)

    def get_config(self):
        config = super(Conv2DX, self).get_config()
        config.pop('rank')
        return config
    
input_layer = Input(shape=(101, 101, 1))
encoder = ZeroPadding2D(padding=((14,14),(14,14))) ( input_layer )
encoder = Conv2D(40, kernel_size=3, activation='relu', padding='same') ( encoder ) 
encoder = Conv2DX(40, kernel_size=3, activation='relu', padding='same') ( encoder ) 
decoder = Conv2D(1, kernel_size=3, activation='sigmoid', padding='same') ( encoder ) 
decoder = Cropping2D(cropping=((14,14),(14,14))) ( decoder )
decoder_instance = Model(input_layer, decoder)
decoder_instance.summary()

def makeGabor( gridSize, angle, lambd, sigmaX, ratioSigmaXY, phase, func=np.cos ):
    if not ( gridSize[0]%2 and gridSize[1]%2 ):
        raise ValueError('makeGabor(): dimensions must be odd numbers.')
    radius = (int(gridSize[0]/2.0), int(gridSize[1]/2.0))
    [x, y] = np.meshgrid(range(-radius[0], radius[0]+1), range(-radius[1], radius[1]+1))
    x1 = x * np.cos(angle) + y * np.sin(angle)
    y1 = -x * np.sin(angle) + y * np.cos(angle)
    return np.exp( - (x1**2 + ratioSigmaXY**2*y1**2)/(2*sigmaX**2)) * func(2*np.pi*(x1/lambd) + phase)

allStructures = np.zeros(( 10, 101, 101))
allStructures[0,:,:] = makeGabor(gridSize=(101,101), angle= 15*np.pi/180, lambd=4.0, sigmaX=2.0, ratioSigmaXY=1.0, phase=0.0, func=np.cos )
allStructures[1,:,:] = makeGabor(gridSize=(101,101), angle= 22*np.pi/180, lambd=4.0, sigmaX=2.0, ratioSigmaXY=1.0, phase=0.0, func=np.cos )
allStructures[2,:,:] = makeGabor(gridSize=(101,101), angle= 45*np.pi/180, lambd=4.0, sigmaX=2.0, ratioSigmaXY=1.0, phase=0.0, func=np.cos )
allStructures[3,:,:] = makeGabor(gridSize=(101,101), angle= 60*np.pi/180, lambd=4.0, sigmaX=2.0, ratioSigmaXY=1.0, phase=0.0, func=np.cos )
allStructures[4,:,:] = makeGabor(gridSize=(101,101), angle= 75*np.pi/180, lambd=4.0, sigmaX=2.0, ratioSigmaXY=1.0, phase=0.0, func=np.cos )
allStructures[5,:,:] = makeGabor(gridSize=(101,101), angle= 90*np.pi/180, lambd=4.0, sigmaX=2.0, ratioSigmaXY=1.0, phase=0.0, func=np.cos )
allStructures[6,:,:] = makeGabor(gridSize=(101,101), angle= 105*np.pi/180, lambd=4.0, sigmaX=2.0, ratioSigmaXY=1.0, phase=0.0, func=np.cos )
allStructures[7,:,:] = makeGabor(gridSize=(101,101), angle= 120*np.pi/180, lambd=4.0, sigmaX=2.0, ratioSigmaXY=1.0, phase=0.0, func=np.cos )
allStructures[8,:,:] = makeGabor(gridSize=(101,101), angle= 135*np.pi/180, lambd=4.0, sigmaX=2.0, ratioSigmaXY=1.0, phase=0.0, func=np.cos )
allStructures[9,:,:] = makeGabor(gridSize=(101,101), angle= 150*np.pi/180, lambd=4.0, sigmaX=2.0, ratioSigmaXY=1.0, phase=0.0, func=np.cos )


testStructures, testStructures, trainStructures, testStructures = \
      train_test_split(allStructures, allStructures, test_size=0.25, random_state=11111)
    
trainStructures = trainStructures.reshape(trainStructures.shape + (1,)) 
testStructures  = testStructures.reshape(testStructures.shape + (1,))

decoder_instance.compile(optimizer='Adadelta', loss='mean_squared_error') 

decoder_instance.fit(trainStructures, trainStructures,
                     epochs=10,
                     batch_size=128,
                     shuffle=True,
                     validation_data=(testStructures, testStructures))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        (None, 101, 101, 1)       0         
_________________________________________________________________
zero_padding2d_30 (ZeroPaddi (None, 129, 129, 1)       0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 129, 129, 40)      400       
_________________________________________________________________
conv2dx_30 (Conv2DX)         (None, 129, 129, 40)      14400     
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 129, 129, 1)       361       
_________________________________________________________________
cropping2d_17 (Cropping2D)   (None, 101, 101, 1)       0         
Total params: 15,161
Trainable params: 15,161
Non-trainable params: 0
_________________________________________________________________
Train 